In [1]:
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
import numpy as np
import pandas as pd

import sys
sys.path.insert(0,'../../')

from rcalgo.tf.utils.text_utils import *
import pickle as pkl
from rcalgo.tf.tfmodels.texar_models.tx_transformerlm import *
import texar.tf as tx
#from texar_models.tx_rnnlm import *



In [2]:
df = pd.read_csv('data/ratings.csv')
df['comment'] = df['comment'].astype(str)
sentences = df['comment'].tolist()
sentences = np.array([c for c in sentences if c != 'nan' and len(c) <= 498])
if len(sentences)%200 != 0:
    sentences = sentences[:-(len(sentences)%200)]
else:
    sentences = sentences    
seqlen = np.array([len(c) for c in sentences])


In [2]:
with open('tmp_photo_comments_sentences.pkl','rb') as f:
    sentences = pkl.load(f)

In [2]:
#对联数据
dir_path = 'data/couplet/{}.txt'
with open(dir_path.format('in'), 'r') as f:
    source = f.readlines()
source = np.array([''.join(s.split()) for s in source])
with open(dir_path.format('out'), 'r') as f:
    target = f.readlines()
target = np.array([''.join(t.split()) for t in target])
sentences = np.array([(s + '。' + t) for s, t in zip(source, target)])
maxlen = np.max([len(s) for s in sentences]) + 2
sentences = sentences[:-(len(sentences)%100)]

In [3]:
word_freq = word_count(sentences, word_seg=False)
term_id_map = build_word2idx(word_freq, freq_threshold=5)
id_term_map = dict([(v,k) for k, v in term_id_map.items()]) 


In [17]:
from rcalgo.tf.utils.train_config import *

args = parse_initial_training_args(
    [
        '--learning-rate', '0.0001',
        '--n-epochs', '4',
        '--opt', 'adam',
        '--batch-size', '512',
        '--p-coef', '0',
        '--keep-prob','0.9',
        '--embedding-size', '512',
        '--model-dir', 'model/',
        '--model-name', 'xxx',
        '--num-layers', '4',
        '--gpu-id', '0',
        '--gpu-num', '1',
        '--hidden-size','512',
        '--maxlen', '50', #str(maxlen),
        '--epoch-save',
        '--use-residual',
        '--position-type', '2,3,4,5',
        '--beam-width', '200',
        '--sample-decay-factor', '5000',
        '--expand-input',
        #'--epoch-save'
       # '--sampling-probability', '0.01',
      #  '--epoch-save',
    ]
)

args.add_conv_flag = False


In [18]:
res = {}
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
#rnnlm_model = TxRNNLMModel(args, graph_to_use, word_dict=term_id_map)
rnnlm_model = TxTransformerLMModel(args, graph_to_use, word_dict=term_id_map)
rnnlm_model.build_model()
rnnlm_model.build_model_summary()
#rnnlm_model.job.dump_checkpoint()
display(rnnlm_model.model_summary())
#rnnlm_model.run([sentences], test_size=0.2)

total parameter number: 18953728


variable_name variable_shape  \
0     gpu/embedding/embedding_layer/embedding_table:0    [6177, 512]   
1                           gpu/position_embedder/w:0      [50, 512]   
2              gpu/transformer_decoder/layer_0/beta:0          [512]   
3             gpu/transformer_decoder/layer_0/gamma:0          [512]   
4   gpu/transformer_decoder/layer_0/self_attention...     [512, 512]   
5   gpu/transformer_decoder/layer_0/self_attention...     [512, 512]   
6   gpu/transformer_decoder/layer_0/self_attention...     [512, 512]   
7   gpu/transformer_decoder/layer_0/self_attention...     [512, 512]   
8   gpu/transformer_decoder/layer_0/past_poswise_l...          [512]   
9   gpu/transformer_decoder/layer_0/past_poswise_l...          [512]   
10  gpu/transformer_decoder/layer_0/ffn/conv1/kern...    [512, 2048]   
11   gpu/transformer_decoder/layer_0/ffn/conv1/bias:0         [2048]   
12  gpu/transformer_decoder/layer_0/ffn/conv2/kern...    [2048, 512]   
13   gpu/transformer_decoder/layer_0/ffn/conv2/bias:0          [512]   
14             gpu/transformer_decoder/layer_1/beta:0          [512]   
15            gpu/transformer_decoder/layer_1/gamma:0          [512]   
16  gpu/transformer_decoder/layer_1/self_attention...     [512, 512]   
17  gpu/transformer_decoder/layer_1/self_attention...     [512, 512]   
18  gpu/transformer_decoder/layer_1/self_attention...     [512, 512]   
19  gpu/transformer_decoder/layer_1/self_attention...     [512, 512]   
20  gpu/transformer_decoder/layer_1/past_poswise_l...          [512]   
21  gpu/transformer_decoder/layer_1/past_poswise_l...          [512]   
22  gpu/transformer_decoder/layer_1/ffn/conv1/kern...    [512, 2048]   
23   gpu/transformer_decoder/layer_1/ffn/conv1/bias:0         [2048]   
24  gpu/transformer_decoder/layer_1/ffn/conv2/kern...    [2048, 512]   
25   gpu/transformer_decoder/layer_1/ffn/conv2/bias:0          [512]   
26             gpu/transformer_decoder/layer_2/beta:0          [512]   
27            gpu/transformer_decoder/layer_2/gamma:0          [512]   
28  gpu/transformer_decoder/layer_2/self_attention...     [512, 512]   
29  gpu/transformer_decoder/layer_2/self_attention...     [512, 512]   
30  gpu/transformer_decoder/layer_2/self_attention...     [512, 512]   
31  gpu/transformer_decoder/layer_2/self_attention...     [512, 512]   
32  gpu/transformer_decoder/layer_2/past_poswise_l...          [512]   
33  gpu/transformer_decoder/layer_2/past_poswise_l...          [512]   
34  gpu/transformer_decoder/layer_2/ffn/conv1/kern...    [512, 2048]   
35   gpu/transformer_decoder/layer_2/ffn/conv1/bias:0         [2048]   
36  gpu/transformer_decoder/layer_2/ffn/conv2/kern...    [2048, 512]   
37   gpu/transformer_decoder/layer_2/ffn/conv2/bias:0          [512]   
38             gpu/transformer_decoder/layer_3/beta:0          [512]   
39            gpu/transformer_decoder/layer_3/gamma:0          [512]   
40  gpu/transformer_decoder/layer_3/self_attention...     [512, 512]   
41  gpu/transformer_decoder/layer_3/self_attention...     [512, 512]   
42  gpu/transformer_decoder/layer_3/self_attention...     [512, 512]   
43  gpu/transformer_decoder/layer_3/self_attention...     [512, 512]   
44  gpu/transformer_decoder/layer_3/past_poswise_l...          [512]   
45  gpu/transformer_decoder/layer_3/past_poswise_l...          [512]   
46  gpu/transformer_decoder/layer_3/ffn/conv1/kern...    [512, 2048]   
47   gpu/transformer_decoder/layer_3/ffn/conv1/bias:0         [2048]   
48  gpu/transformer_decoder/layer_3/ffn/conv2/kern...    [2048, 512]   
49   gpu/transformer_decoder/layer_3/ffn/conv2/bias:0          [512]   
50                     gpu/transformer_decoder/beta:0          [512]   
51                    gpu/transformer_decoder/gamma:0          [512]   
52             gpu/transformer_decoder/dense/kernel:0    [512, 6178]   

   parameters  
0     3162624  
1       25600  
2         512  
3         512  
4      262144  
5      262144  
6      262144  
7      262144  
8      

In [20]:
def out_print(num_arr, id_term_map, eos=rnnlm_model.nb_words - 1):
    #print(num_arr)
    text = ''
    for i in num_arr:
        if i >= eos-100:
            break
        else:
            text += id_term_map[i]
    return text

In [21]:
with rnnlm_model.job.session.graph.as_default():
    input_var, context_var = tf.get_collection('input_dict')
    go_value = tf.get_collection('go_value')[0]
    outputs = tf.get_collection('output_dict')[0:5]
    greedy_logits, greedy_sample_id, sample_logits, sample_sample_id, beam_search_results = outputs
    top_k = tf.get_collection('top_k_sample')[0]
    softmax_temperature = tf.get_collection('softmax_temperature')[0]


In [53]:
sents = np.random.choice(source, 1, replace=False)
samples = rnnlm_model.job.session.run(sample_sample_id, 
                                      feed_dict={input_var:sents, context_var:sents, top_k: 10})
print(out_print(samples[0][1:], id_term_map))

峰起岭以南，鼎峙三峰，仍是中峰尤俊拔。云游峰而外，风来七月，但凭大笔更风骚


In [27]:
samples = rnnlm_model.job.session.run([sample_sample_id], feed_dict={input_var:[''], context_var:[''], top_k: 10})
print(out_print(samples[0][0][1:], id_term_map))

一叶归根风扫地，谁能上下。十年落木雨滋人，哪有东西


In [13]:
samples = rnnlm_model.job.session.run([beam_search_results], feed_dict={input_var:[''], context_var:[''], top_k: 10})
for i in range(200):
    print(out_print(samples[0][0,:,i], id_term_map))

一身正气。两袖清风
平天湖上赏明月。杏花村里听春风
平天湖上赏明月。杏花村里笑春风
金猴献瑞。玉兔迎春
龙腾虎跃。虎跃龙腾
平天湖上赏明月。杏花村里话春风
龙腾盛世。蛇舞丰年
平天湖上赏明月。杏花村里赏春风
平天湖上赏明月。杏花村里醉春风
平天湖上赏明月。杏花村里看春风
平天湖上赏明月。杏花村里迎春风
平天湖上赏明月。杏花村里寻春风
平天湖上赏明月。杏花村里觅知音
平天湖上赏明月。杏花村里话春秋
平天湖上赏明月。杏花村里弄春风
平天湖上赏明月。杏花村里待春风
山清水秀地干净。鸟语花香天井红
平天湖上赏明月。杏花村里听清风
平天湖上赏明月。杏花村里醉清风
平天湖上赏明月。杏花村里看桃花
平天湖上赏明月。杏花村里迎新春
平天湖上赏明月。杏花村里觅佳人
平天湖上赏明月。杏花村里送春风
春风拂柳绿。夏雨润花红
平天湖上赏明月。杏花村里探春风
平天湖上赏明月。杏花村里有春风
平天湖上赏明月。杏花村里戏春风
平天湖上赏明月。杏花村里弄清风
平天湖上赏明月。杏花村里赏佳人
平天湖上赏明月。杏花村里话丰年
平天湖上赏明月。杏花村里闻香风
平天湖上赏明月。杏花村里画春风
平天湖上赏明月。杏花村里贺新春
平天湖上赏明月。杏花村里戏清风
平天湖上赏明月。杏花村里度春风
平天湖上赏明月。杏花村里赏清风
平天湖上赏明月。杏花村里得春风
平天湖上赏明月。杏花村里寻芳邻
平天湖上赏明月。杏花村里赏桃花
平天湖上赏明月。杏花村里寻芳春
平天湖上赏明月。杏花村里播春风
平天湖上赏明月。杏花村里观春风
平天湖上赏明月。杏花村里看清风
平天湖上赏明月。杏花村里舞春风
山清水秀地干净。国富民强民富强
平天湖上赏明月。杏花村里有佳人
平天湖上赏明月。杏花村里读春风
平天湖上赏明月。杏花村里觅春风
平天湖上赏明月。杏花村里沐春风
平天湖上赏明月。杏花村里种春风
平天湖上赏明月。杏花村里听雨声
平天湖上赏明月。杏花村里寻芳菲
平天湖上赏明月。杏花村里读春秋
平天湖上赏明月。杏花村里唱春风
平天湖上赏明月。杏花村里逢春风
平天湖上赏明月。杏花村里读好书
平天湖上赏明月。杏花村里赏春光
平天湖上赏明月。杏花村里放春风
山清水秀地干净。国泰民安民富强
平天湖上赏明月。杏花村里煮春风
平天湖上赏明月。杏花村里品清香
平天湖上赏明月。杏花村里看飞花
平天湖上赏明月。杏花村里赏美人
平天湖上赏明月。杏花村里看春光
平天湖上